# Przetwarzanie wstępne. Filtracja kontekstowa.


### Cel:
- zapoznanie z pojęciem kontekstu / filtracji kontekstowej,
- zapoznanie z pojęciem konwolucji (splotu),
- zapoznanie z wybranymi filtrami:
	- filtry liniowe dolnoprzepustowe:
		- filtr uśredniający,
		- filtr Gaussa.
	- filtry nielinowe:
		- mediana,
		- mediana dla obrazów kolorowych.
	- filtry liniowe górnoprzepustowe:
			- laplasjan,
			- operator Robersta, Prewitta, Sobela.
- zadanie domowe: adaptacyjna filtracja medianowa.

### Filtry liniowe uśredniające (dolnoprzepustowe)

Jest to podstawowa rodzina filtrów stosowana w cyfrowym przetwarzaniu obrazów. 
Wykorzystuje się je w celu "rozmazania" obrazu i tym samym redukcji szumów (zakłóceń) na obrazie.
Filtr określony jest przez dwa parametry: rozmiar maski (ang. _kernel_) oraz wartości współczynników maski.

Warto zwrócić uwagę, że omawiane w niniejszym rozdziale operacje generują nową wartość piksela na podstawie pewnego fragmentu obrazu (tj. kontekstu), a nie jak operacje punktowe tylko na podstawie jednego piksela.


1. Wczytaj obraz _plansza.png_.
W dalszej części ćwiczenia sprawdzenie działania filtracji dla innych obrazów sprowadzi się do wczytania innego pliku.

2. Podstawowa funkcja to `cv2.filter2D`  - realizacja filtracji konwolucyjnej.
   Proszę sprawdzić jej dokumentację i zwrócić uwagę na obsługę problemu brzegowego (na krawędziach istnieją piksele dla których nie da się wyznaczyć otoczenia).

  Uwaga. Problem ten można też rozwiązać z użyciem funkcji `signal.convolve2d` z biblioteki _scipy_ (`from scipy import signal`).

3. Stwórz podstawowy filtr uśredniający o rozmiarze $3 \times 3$ -- za pomocą funkcji `np.ones`. Wykonaj konwolucję na wczytanym obrazie. Na wspólnym rysunku wyświetl obraz oryginalny, po filtracji oraz moduł z różnicy.

4. Przeanalizuj otrzymane wyniki. Jakie elementy zawiera obraz "moduł z różnicy"? Co na tej podstawie można powiedzieć o filtracji dolnoprzepustowej?

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from scipy import signal


# Obrazki
if not os.path.exists("jet.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/jet.png --no-check-certificate
if not os.path.exists("kw.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/kw.png --no-check-certificate
if not os.path.exists("moon.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/moon.png --no-check-certificate
if not os.path.exists("lenaSzum.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/lenaSzum.png --no-check-certificate
if not os.path.exists("lena.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/lena.png --no-check-certificate
if not os.path.exists("plansza.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/plansza.png --no-check-certificate



In [ ]:
plansza = cv2.imread('plansza.png')          
plansza = cv2.cvtColor(plansza, cv2.COLOR_BGR2GRAY) 
filter = np.ones((3,3), np.float32)/9
dst = cv2.filter2D(plansza, -1, filter)
diff=cv2.subtract(plansza, dst)
plt.imshow(plansza, 'gray')
plt.show()
plt.imshow(dst, 'gray')
plt.show()
plt.imshow(diff, 'gray')
plt.show()

5. Na wspólnym rysunku wyświetl wyniki filtracji uśredniającej z oknem o rozmiarze 3, 5, 9, 15 i 35. 
Wykorzystaj polecenie `plt.subplot`. 
Przeanalizuj wpływ rozmiaru maski na wynik. 

In [ ]:
def filter(size, img):
    a = np.ones((size,size), np.float32)/(size**2)
    filteredImg = cv2.filter2D(img, -1, a)
    return filteredImg
plansza = cv2.imread('plansza.png')          
plansza = cv2.cvtColor(plansza, cv2.COLOR_BGR2GRAY)
fig, axs = plt.subplots(6, figsize=(20,20))
axs[0].imshow(plansza, 'gray')
axs[0].axis('off')
axs[1].imshow(filter(3, plansza), 'gray')
axs[1].axis('off')
axs[2].imshow(filter(5, plansza), 'gray')
axs[2].axis('off')
axs[3].imshow(filter(9, plansza), 'gray')
axs[3].axis('off')
axs[4].imshow(filter(15, plansza), 'gray')
axs[4].axis('off')
axs[5].imshow(filter(35, plansza), 'gray')
axs[5].axis('off')


6. Wczytaj obraz _lena.png_.
Zaobserwuj efekty filtracji dolnoprzepustowej dla obrazu rzeczywistego.

In [ ]:
lena = cv2.imread('lena.png')          
lena = cv2.cvtColor(lena, cv2.COLOR_BGR2GRAY)
fig, axs = plt.subplots(6, figsize=(20,20))
axs[0].imshow(lena, 'gray')
axs[0].axis('off')
axs[1].imshow(filter(3, lena), 'gray')
axs[1].axis('off')
axs[2].imshow(filter(5, lena), 'gray')
axs[2].axis('off')
axs[3].imshow(filter(9, lena), 'gray')
axs[3].axis('off')
axs[4].imshow(filter(15, lena), 'gray')
axs[4].axis('off')
axs[5].imshow(filter(35, lena), 'gray')
axs[5].axis('off')


7. Niekorzystny efekt towarzyszący wykonanym filtracjom dolnoprzepustowym to utrata ostrości. 
Częściowo można go zniwelować poprzez odpowiedni dobór maski. 
Wykorzystaj maskę:  `M = np.array([1 2 1; 2 4 2; 1 2 1])`. 
Przed obliczeniami należy jeszcze wykonać normalizację - podzielić każdy element maski przez sumę wszystkich elementów: `M = M/sum(sum(M));`.
Tak przygotowaną maskę wykorzystaj w konwolucji - wyświetl wyniki tak jak wcześniej.
Możliwe jest też wykorzystywanie innych masek - współczynniki można dopasowywać do konkretnego problemu.

In [ ]:
def filter2(size, img, mask):
    dst = cv2.filter2D(img, -1, mask)
    return dst
M = np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]])
M = M/sum(sum(M));
fig, axs = plt.subplots(6, figsize=(20,20))
axs[0].imshow(lena, 'gray')
axs[0].axis('off')
axs[1].imshow(filter2(3, lena, M), 'gray')
axs[1].axis('off')
axs[2].imshow(filter2(5, lena, M), 'gray')
axs[2].axis('off')
axs[3].imshow(filter2(9, lena, M), 'gray')
axs[3].axis('off')
axs[4].imshow(filter2(15, lena, M), 'gray')
axs[4].axis('off')
axs[5].imshow(filter2(35, lena, M), 'gray')
axs[5].axis('off')


8. Skuteczną i często wykorzystywaną maską jest tzw. maska Gasussa.
Jest to zbiór liczb, które aproksymują dwuwymiarowy rozkład Gaussa. 
Parametrem jest odchylenie standardowe i rozmiar maski.

9. Wykorzystując przygotowaną funkcję `fgaussian` stwórz maskę o rozmiarze $5 \times 5$ i odchyleniu standardowym 0.5.
  Wykorzystując funkcję `mesh` zwizualizuj filtr.
  Sprawdź jak parametr ``odchylenie standardowe'' wpływa na ``kształt'' filtru.

  Uwaga. W OpenCV dostępna jest *dedykowana* funkcja do filtracji Gaussa - `GaussianBlur`.
  Proszę na jednym przykładzie porównać jej działanie z użytym wyżej rozwiązaniem.

10. Wykonaj filtrację dla wybranych (2--3) wartości odchylenia standardowego.


In [ ]:
def fgaussian(size, sigma):
     m = n = size
     h, k = m//2, n//2
     x, y = np.mgrid[-h:h+1, -k:k+1]
     g = np.exp(-(x**2 + y**2)/(2*sigma**2))
     return g /g.sum() 
    
    
def mesh(fun, size):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    

    X = np.arange(-size//2, size//2, 1)
    Y = np.arange(-size//2, size//2, 1)
    X, Y = np.meshgrid(X, Y)
    Z = fun
    
    ax.plot_surface(X, Y, Z)
    
    plt.show()
    
mask = fgaussian(5, 0.5)
mesh(mask, 5)
plt.imshow(lena, 'gray')
plt.show()
plt.imshow(filter2(5, lena, mask), 'gray')
plt.show()
blur = cv2.GaussianBlur(lena,(5,5),0.5)
plt.imshow(blur, 'gray')
mask = fgaussian(5, 1)
mesh(mask, 5)
plt.imshow(lena, 'gray')
plt.show()
plt.imshow(filter2(5, lena, mask), 'gray')
plt.show()
blur = cv2.GaussianBlur(lena,(5,5),1)
plt.imshow(blur, 'gray')
mask = fgaussian(5, 2)
mesh(mask, 5)
plt.imshow(lena, 'gray')
plt.show()
plt.imshow(filter2(5, lena, mask), 'gray')
plt.show()
blur = cv2.GaussianBlur(lena,(5,5),2)
plt.imshow(blur, 'gray')

### Filtry nieliniowe -- mediana

Filtry rozmywające redukują szum, ale niekorzystnie wpływają na ostrość obrazu.
Dlatego często wykorzystuje się filtry nieliniowe - np. filtr medianowy (dla przypomnienia: mediana - środkowa wartość w posortowanym ciągu liczb).

Podstawowa różnica pomiędzy filtrami liniowymi, a nieliniowymi polega na tym, że przy filtracji liniowej na nową wartość piksela ma wpływ wartość wszystkich pikseli z otoczenia (np. uśrednianie, czasem ważone), natomiast w przypadku filtracji nieliniowej jako nowy piksel wybierana jest któraś z wartości otoczenia - według jakiegoś wskaźnika (wartość największa, najmniejsza czy właśnie mediana).


1. Wczytaj obraz _lenaSzum.png_ (losowe 10% pikseli białych lub czarnych - tzw. zakłócenia impulsowe). Przeprowadź filtrację uśredniającą z rozmiarem maski 3x3. Wyświetl, podobnie jak wcześniej, oryginał, wynik filtracji i moduł z różnicy. Wykorzystując funkcję ``cv2.medianBlur` wykonaj filtrację medianową _lenaSzum.png_ (z rozmiarem maski $3 \times 3$). Wyświetl, podobnie jak wcześniej, oryginał, wynik filtracji i moduł z różnicy. Która filtracja lepiej radzi sobie z tego typu szumem?

  Uwaga. Taki sam efekt da również użycie funkcji `signal.medfilt2d`.


In [ ]:
lena = cv2.imread('lenaSzum.png')          
lena = cv2.cvtColor(lena, cv2.COLOR_BGR2GRAY)
median = cv2.medianBlur(lena,3)
diff=cv2.subtract(lena, median)
plt.imshow(lena, 'gray')
plt.show()
plt.imshow(median, 'gray')
plt.show()
plt.imshow(diff, 'gray')
plt.show()

In [ ]:
#2. Przeprowadź filtrację uśredniającą, a następnie medianową obrazu _lena.png_.
#   Wyniki porównaj - dla obu wyświetl: oryginał, wynik filtracji i moduł z różnicy.
#   Szczególną uwagę zwróć na ostrość i krawędzie.
#   W której filtracji krawędzie zostają lepiej zachowane?
lena = cv2.imread('lena.png')          
lena = cv2.cvtColor(lena, cv2.COLOR_BGR2GRAY)
blur = cv2.blur(lena,(3,3))
median = cv2.medianBlur(lena,3)
diff=cv2.subtract(lena, blur)
plt.imshow(lena, 'gray')
plt.show()
plt.imshow(blur, 'gray')
plt.show()
plt.imshow(diff, 'gray')
plt.show()
diff=cv2.subtract(lena, median)
plt.imshow(median, 'gray')
plt.show()
plt.imshow(diff, 'gray')
plt.show()

2. Przeprowadź filtrację uśredniającą, a następnie medianową obrazu _lena.png_.
   Wyniki porównaj - dla obu wyświetl: oryginał, wynik filtracji i moduł z różnicy.
   Szczególną uwagę zwróć na ostrość i krawędzie.
   W której filtracji krawędzie zostają lepiej zachowane?

3. Ciekawy efekt można uzyskać wykonując filtrację medianową wielokrotnie. Określa się go mianem  posteryzacji.  W wyniku przetwarzania z obrazka usunięte zostają detale, a duże obszary uzyskują tą samą wartość jasności.  Wykonaj operację mediany $5 \times 5$ na obrazie _lena.png_ 10-krotnie. (wykorzystaj np. pętlę `for`).


Inne filtry nieliniowe:
- filtr modowy - moda (dominanta) zamiast mediany,
- filtr olimpijski - średnia z podzbioru otoczenia (bez wartości ekstremalnych),
- hybrydowy filtr medianowy - mediana obliczana osobno w różnych podzbiorach otoczenia (np. kształt ``x'',``+''), a jako wynik brana jest mediana ze zbioru wartość elementu centralnego, mediana z ``x'' i mediana z ``+'',
- filtr minimalny i maksymalny (będą omówione przy okazji operacji morfologicznych w dalszej części kursu).


Warto zdawać sobie sprawę, z szerokich możliwości dopasowywania rodzaju filtracji do konkretnego rozważanego problemu i rodzaju zaszumienia występującego na obrazie.

In [ ]:
lena = cv2.imread('lena.png')          
lena = cv2.cvtColor(lena, cv2.COLOR_BGR2GRAY)
median = cv2.medianBlur(lena,5)
for i in range(1,10):
    median = cv2.medianBlur(median,5)
plt.imshow(lena, 'gray')
plt.show()
plt.imshow(median, 'gray')
plt.show()



## Filtry liniowe górnoprzepustowe (wyostrzające, wykrywające krawędzie)

Zadaniem filtrów górnoprzepustowych jest wydobywanie z obrazu składników odpowiedzialnych za szybkie zmiany jasności - konturów, krawędzi, drobnych elementów tekstury.

### Laplasjan (wykorzystanie drugiej pochodnej obrazu)

1. Wczytaj obraz _moon.png_.

2. Wprowadź podstawową maskę laplasjanu:
\begin{equation}
M = 
\begin{bmatrix}
0 & 1& 0 \\ 1 & -4 & 1 \\ 0 & 1 & 0
\end{bmatrix}
\end{equation}

3. Przed rozpoczęciem obliczeń należy dokonać normalizacji maski - dla rozmiaru $3 \times 3$ podzielić każdy element przez 9.
   Proszę zwrócić uwagę, że nie można tu zastosować takiej samej normalizacji, jak dla filtrów dolnoprzepustowanych, gdyż skutkowałby to dzieleniem przez 0.

4. Wykonaj konwolucję obrazu z maską (`c2.filter2D`). Przed wyświetleniem, wynikowy obraz należy poddać normalizacji (występują ujemne wartości). Najczęściej wykonuje się jedną z dwóch operacji:
- skalowanie (np. poprzez dodatnie 128 do każdego z pikseli),
- moduł (wartość bezwzględna).

Wykonaj obie normalizacje. 
Na wspólnym wykresie wyświetl obraz oryginalny oraz przefiltrowany po obu normalizacjach. 

In [ ]:
moon = cv2.imread('moon.png')
moon = cv2.cvtColor(moon, cv2.COLOR_BGR2GRAY)
M = (np.matrix('0 1 0; 1 -4 1; 0 1 0') / 9)
plt.imshow(moon, 'gray')
plt.show()
dst = abs(cv2.filter2D(moon, -1, M))
plt.imshow(dst, 'gray')

7. Efekt wyostrzenia uzyskuje się po odjęciu/dodaniu (zależy do maski) rezultatu filtracji laplasjanowej i oryginalnego obrazu. Wyświetl na jednym wykresie: obraz oryginalny, sumę oryginału i wyniku filtracji oraz różnicę (bezwzględną) oryginału i wyniku filtracji.
 Uwaga. Aby uniknąć artefaktów, należy obraz wejściowy przekonwertować do formatu ze znakiem.



In [ ]:
plt.imshow(moon, 'gray')
plt.show()
diff=cv2.subtract(moon, dst)
plt.imshow(diff, 'gray')
plt.show()
con = cv2.add(moon, dst)
plt.imshow(con, 'gray')
plt.show()

### Gradienty (wykorzystanie pierwszej pochodnej obrazu)

1. Wczytaj obraz _kw.png_. Stwórz odpowiednie maski opisane w kolejnych punktach i dokonaj filtracji.
2. Wykorzystując gradient Robertsa przeprowadź detekcję krawędzi - poprzez wykonanie konwolucji obrazu z daną maską:
\begin{equation}
R1 = \begin{bmatrix} 0 & 0 & 0 \\ -1 & 0 & 0 \\ 0 & 1 & 0 \end{bmatrix}   
R2 = \begin{bmatrix} 0 & 0 & 0 \\ 0 & 0 & -1 \\ 0 & 1 & 0 \end{bmatrix}
\end{equation}

Wykorzystaj stworzony wcześniej kod (przy laplasjanie) - dwie metody normalizacji oraz sposób wyświetlania.

3. Analogicznie przeprowadź detekcję krawędzi za pomocą gradientu Prewitta (pionowy i poziomy)
\begin{equation}
P1 = \begin{bmatrix} -1 & 0 & 1 \\ -1 & 0 & 1 \\ -1 & 0 & 1 \end{bmatrix}   
P2 = \begin{bmatrix} -1 & -1 & -1 \\ 0 & 0 & 0 \\ 1 & 1 & 1 \end{bmatrix}
\end{equation}

4. Podobnie skonstruowany jest gradient Sobela (występuje osiem masek, zaprezentowane są dwie ``prostopadłe''):
\begin{equation}
S1 = \begin{bmatrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{bmatrix}   
S2 = \begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ 1 & 2 & 1 \end{bmatrix}
\end{equation}

Przeprowadź detekcję krawędzi za pomocą gradientu Sobela. 

In [ ]:
from skimage import filters

kw = cv2.imread('kw.png')
kw = cv2.cvtColor(kw, cv2.COLOR_BGR2GRAY)
edge_roberts = filters.roberts(kw)
edge_sobel = filters.sobel(kw)
edge_prewitt = filters.prewitt(kw)
plt.imshow(kw, 'gray')
plt.show()
plt.imshow(edge_roberts, 'gray')
plt.show()
plt.imshow(edge_sobel, 'gray')
plt.show()
plt.imshow(edge_prewitt, 'gray')
plt.show()
R1 = (np.matrix('0 0 0; -1 0 0; 0 1 0') / 9)
R2 = (np.matrix('0 0 0; 0 0 -1; 0 1 0') / 9)
P1 = (np.matrix('-1 0 1; -1 0 1; -1 0 1') / 9)
P2 = (np.matrix('-1 -1 -1; 0 0 0; 1 1 1') / 9)
S1 = (np.matrix('-1 0 1; -2 0 2; -1 0 1') / 9)
S2 = (np.matrix('-1 -2 -1; 0 0 0; 1 2 1') / 9)
dstr1 = abs(cv2.filter2D(kw, -1, R1))
dstr2 = abs(cv2.filter2D(kw, -1, R2))
dstp1 = abs(cv2.filter2D(kw, -1, P1))
dstp2 = abs(cv2.filter2D(kw, -1, P2))
dsts1 = abs(cv2.filter2D(kw, -1, S1))
dsts2 = abs(cv2.filter2D(kw, -1, S2))



5. Na podstawie dwóch ortogonalnych masek np. Sobela można stworzyć tzw. filtr kombinowany - pierwiastek kwadratowy z sumy kwadratów gradientów:
\begin{equation}
OW = \sqrt{(O * S1)^2 + (O * S2)^2}
\end{equation}
gdzie:  $OW$ - obraz wyjściowy, $O$ - obraz oryginalny (wejściowy), $S1,S2$ - maski Sobela, $*$ - operacja konwolucji.

Zaimplementuj filtr kombinowany.

Uwaga. Proszę zwrócić uwagę na konieczność zmiany formatu danych obrazu wejściowego - na typ znakiem



In [ ]:
kw = cv2.imread('kw.png')
kw = cv2.cvtColor(kw, cv2.COLOR_BGR2GRAY)
S1 = (np.matrix('-1 0 1; -2 0 2; -1 0 1') / 9)
S2 = (np.matrix('-1 -2 -1; 0 0 0; 1 2 1') / 9)
OW = (np.add((cv2.filter2D(kw, -1, S1)**2),(cv2.filter2D(kw, -1, S2)**2)))**(1/2)
plt.imshow(OW, 'gray')
plt.show()


6. Istnieje alternatywna wersja filtra kombinowanego, która zamiast pierwiastka z sumy kwadratów wykorzystuje sumę modułów (prostsze obliczenia). 
Zaimplementuj tę wersję. 

In [ ]:
kw = cv2.imread('kw.png')
kw = cv2.cvtColor(kw, cv2.COLOR_BGR2GRAY)
S1 = (np.matrix('-1 0 1; -2 0 2; -1 0 1') / 9)
S2 = (np.matrix('-1 -2 -1; 0 0 0; 1 2 1') / 9)
OW = np.add(abs(cv2.filter2D(kw, -1, S1)**2),abs(cv2.filter2D(kw, -1, S2)**2))
plt.imshow(OW, 'gray')
plt.show()

7. Wczytaj plik _jet.png_ (zamiast _kw.png_).
Sprawdź działanie obu wariantów filtracji kombinowanej.

In [ ]:
kw = cv2.imread('jet.png')
kw = cv2.cvtColor(kw, cv2.COLOR_BGR2GRAY)
S1 = (np.matrix('-1 0 1; -2 0 2; -1 0 1') / 9)
S2 = (np.matrix('-1 -2 -1; 0 0 0; 1 2 1') / 9)
OW1 = (np.add((cv2.filter2D(kw, -1, S1)**2),(cv2.filter2D(kw, -1, S2)**2)))**(1/2)
OW2 = np.add(abs(cv2.filter2D(kw, -1, S1)**2),abs(cv2.filter2D(kw, -1, S2)**2))
plt.imshow(OW1, 'gray')
plt.show()
plt.imshow(OW2, 'gray')
plt.show()